<a href="https://colab.research.google.com/github/T-Sunm/Learn-Feature-Engineering-in-Kaggle/blob/main/Exercise_Creating_Features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'fe-course-data:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F933090%2F1828856%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240923%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240923T152008Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D27401332d35f8aa9f55f94a49274357344c95e4b4bcc3b7de685a4a419f47574701b35f80ed138b19a1d1982a97d4fb2f73d03e69adbec606b9e9386bfd2bad37831483637a1d604503413fb37baa3760fe5db84479807bc5137183bf515f6345a93c72467b4d327f333085eeee52182b70eb78fcd7f470f735ac29f8695be321113790d795b6b18b8d4556c4b7abd180cc3702fd0434db2c947b33ff9c90e18ff5e24470bde7266d80afa5dabcd841cab14c5520b838ee9f99a7ecee12896ef3dac8f7b860c8182b4215ddf6e0bbf6c54ba4a41c3fe646b2ad6e9e492f106ee99c59f36d94097b897259368625edd546ce78271e6020aa6a33146c9348a6fc9'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


**This notebook is an exercise in the [Feature Engineering](https://www.kaggle.com/learn/feature-engineering) course.  You can reference the tutorial at [this link](https://www.kaggle.com/ryanholbrook/creating-features).**

---


# Introduction #

In this exercise you'll start developing the features you identified in Exercise 2 as having the most potential. As you work through this exercise, you might take a moment to look at the data documentation again and consider whether the features we're creating make sense from a real-world perspective, and whether there are any useful combinations that stand out to you.

Run this cell to set everything up!

In [ ]:
# Setup feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.feature_engineering_new.ex3 import *

import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score
from xgboost import XGBRegressor


def score_dataset(X, y, model=XGBRegressor()):
    # Label encoding for categoricals
    for colname in X.select_dtypes(["category", "object"]):
        X[colname], _ = X[colname].factorize()
    # Metric for Housing competition is RMSLE (Root Mean Squared Log Error)
    score = cross_val_score(
        model, X, y, cv=5, scoring="neg_mean_squared_log_error",
    )
    score = -1 * score.mean()
    score = np.sqrt(score)
    return score


# Prepare data
df = pd.read_csv("../input/fe-course-data/ames.csv")
X = df.copy()
y = X.pop("SalePrice")

-------------------------------------------------------------------------------

Let's start with a few mathematical combinations. We'll focus on features describing areas -- having the same units (square-feet) makes it easy to combine them in sensible ways. Since we're using XGBoost (a tree-based model), we'll focus on ratios and sums.

# 1) Create Mathematical Transforms

Create the following features:

- `LivLotRatio`: the ratio of `GrLivArea` to `LotArea`
- `Spaciousness`: the sum of `FirstFlrSF` and `SecondFlrSF` divided by `TotRmsAbvGrd`
- `TotalOutsideSF`: the sum of `WoodDeckSF`, `OpenPorchSF`, `EnclosedPorch`, `Threeseasonporch`, and `ScreenPorch`

In [ ]:
X.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YearSold,SaleType,SaleCondition
0,One_Story_1946_and_Newer_All_Styles,Residential_Low_Density,141.0,31770.0,Pave,No_Alley_Access,Slightly_Irregular,Lvl,AllPub,Corner,...,0.0,0.0,No_Pool,No_Fence,NaN,0.0,5,2010,WD,Normal
1,One_Story_1946_and_Newer_All_Styles,Residential_High_Density,80.0,11622.0,Pave,No_Alley_Access,Regular,Lvl,AllPub,Inside,...,120.0,0.0,No_Pool,Minimum_Privacy,NaN,0.0,6,2010,WD,Normal
2,One_Story_1946_and_Newer_All_Styles,Residential_Low_Density,81.0,14267.0,Pave,No_Alley_Access,Slightly_Irregular,Lvl,AllPub,Corner,...,0.0,0.0,No_Pool,No_Fence,Gar2,12500.0,6,2010,WD,Normal
3,One_Story_1946_and_Newer_All_Styles,Residential_Low_Density,93.0,11160.0,Pave,No_Alley_Access,Regular,Lvl,AllPub,Corner,...,0.0,0.0,No_Pool,No_Fence,NaN,0.0,4,2010,WD,Normal
4,Two_Story_1946_and_Newer,Residential_Low_Density,74.0,13830.0,Pave,No_Alley_Access,Slightly_Irregular,Lvl,AllPub,Inside,...,0.0,0.0,No_Pool,Minimum_Privacy,NaN,0.0,3,2010,WD,Normal


`LivLotRatio`
Ý nghĩa: Tỷ lệ này cho biết bao nhiêu phần trăm của lô đất được sử dụng để xây dựng không gian sinh hoạt.
- Tỷ lệ cao có thể cho thấy lô đất nhỏ nhưng ngôi nhà lớn, có thể chỉ ra sự sử dụng tối đa diện tích lô đất.
- Tỷ lệ thấp có thể cho thấy ngôi nhà chiếm một phần nhỏ của lô đất, có nhiều diện tích sân vườn hoặc không gian ngoài trời.

`Spaciousness` đo lường mức độ rộng rãi của từng phòng trong ngôi nhà. Nó phản ánh diện tích trung bình của mỗi phòng trên mặt đất.

In [ ]:
# YOUR CODE HERE
X_1 = pd.DataFrame()  # dataframe to hold new features

X_1["LivLotRatio"] = X['GrLivArea'] / X['LotArea']
X_1["Spaciousness"] = (X['FirstFlrSF'] + X['SecondFlrSF']) / X['TotRmsAbvGrd']
X_1["TotalOutsideSF"] = (X['WoodDeckSF'] + X['OpenPorchSF'] + X['EnclosedPorch'] + X['Threeseasonporch'] + X['ScreenPorch'])


# Check your answer
q_1.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [ ]:
# Lines below will give you a hint or solution code
#q_1.hint()
#q_1.solution()

-------------------------------------------------------------------------------

If you've discovered an interaction effect between a numeric feature and a categorical feature, you might want to model it explicitly using a one-hot encoding, like so:

```
# One-hot encode Categorical feature, adding a column prefix "Cat"
X_new = pd.get_dummies(df.Categorical, prefix="Cat")

# Multiply row-by-row
X_new = X_new.mul(df.Continuous, axis=0)

# Join the new features to the feature set
X = X.join(X_new)
```

# 2) Interaction with a Categorical

We discovered an interaction between `BldgType` and `GrLivArea` in Exercise 2. Now create their interaction features.

- bạn muốn xem xét xem tác động của GrLivArea lên giá trị nhà thay đổi thế nào tùy thuộc vào loại tòa nhà (BldgType). xài hàm get_dummies() để tạo 0-1, hàm mul để nhân giữa Dataframe/Series

In [ ]:
X_2 = pd.get_dummies(df.BldgType, prefix="Type")
X_2.head(3)

,Type_Duplex,Type_OneFam,Type_Twnhs,Type_TwnhsE,Type_TwoFmCon
0,False,True,False,False,False
1,False,True,False,False,False
2,False,True,False,False,False


In [ ]:
X_2.mul(df.GrLivArea, axis = 0).head(3)

,Bldg_Duplex,Bldg_OneFam,Bldg_Twnhs,Bldg_TwnhsE,Bldg_TwoFmCon
0,0.0,2742336.0,0.0,0.0,0.0
1,0.0,802816.0,0.0,0.0,0.0
2,0.0,1766241.0,0.0,0.0,0.0


In [ ]:
# YOUR CODE HERE
# One-hot encode BldgType. Use `prefix="Bldg"` in `get_dummies`
X_2 = pd.get_dummies(df.BldgType, prefix="Bldg")
# Multiply
X_2 = X_2.mul(df.GrLivArea, axis = 0)


# Check your answer
q_2.check()
X_2

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

,Bldg_Duplex,Bldg_OneFam,Bldg_Twnhs,Bldg_TwnhsE,Bldg_TwoFmCon
0,0.0,1656.0,0.0,0.0,0.0
1,0.0,896.0,0.0,0.0,0.0
2,0.0,1329.0,0.0,0.0,0.0
3,0.0,2110.0,0.0,0.0,0.0
4,0.0,1629.0,0.0,0.0,0.0
...,...,...,...,...,...
2925,0.0,1003.0,0.0,0.0,0.0
2926,0.0,902.0,0.0,0.0,0.0
2927,0.0,970.0,0.0,0.0,0.0
2928,0.0,1389.0,0.0,0.0,0.0


In [ ]:
# Lines below will give you a hint or solution code
#q_2.hint()
#q_2.solution()

# 3) Count Feature

Let's try creating a feature that describes how many kinds of outdoor areas a dwelling has. Create a feature `PorchTypes` that counts how many of the following are greater than 0.0:

```
WoodDeckSF
OpenPorchSF
EnclosedPorch
Threeseasonporch
ScreenPorch
```

In [ ]:
X_3 = pd.DataFrame()

component = ['WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', 'Threeseasonporch', 'ScreenPorch']
# YOUR CODE HERE
X_3["PorchTypes"] = df[component].gt(0.0).sum(axis = 1)


# Check your answer
q_3.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [ ]:
# Lines below will give you a hint or solution code
#q_3.hint()
#q_3.solution()

# 4) Break Down a Categorical Feature

`MSSubClass` describes the type of a dwelling:

In [ ]:
df.MSSubClass.unique()

array(['One_Story_1946_and_Newer_All_Styles', 'Two_Story_1946_and_Newer',
       'One_Story_PUD_1946_and_Newer',
       'One_and_Half_Story_Finished_All_Ages', 'Split_Foyer',
       'Two_Story_PUD_1946_and_Newer', 'Split_or_Multilevel',
       'One_Story_1945_and_Older', 'Duplex_All_Styles_and_Ages',
       'Two_Family_conversion_All_Styles_and_Ages',
       'One_and_Half_Story_Unfinished_All_Ages',
       'Two_Story_1945_and_Older', 'Two_and_Half_Story_All_Ages',
       'One_Story_with_Finished_Attic_All_Ages',
       'PUD_Multilevel_Split_Level_Foyer',
       'One_and_Half_Story_PUD_All_Ages'], dtype=object)

You can see that there is a more general categorization described (roughly) by the first word of each category. Create a feature containing only these first words by splitting `MSSubClass` at the first underscore `_`. (Hint: In the `split` method use an argument `n=1`.)

In [ ]:
df["MSSubClass"].str.split('_')

0       [One, Story, 1946, and, Newer, All, Styles]
1       [One, Story, 1946, and, Newer, All, Styles]
2       [One, Story, 1946, and, Newer, All, Styles]
3       [One, Story, 1946, and, Newer, All, Styles]
4                    [Two, Story, 1946, and, Newer]
                           ...                     
2925                        [Split, or, Multilevel]
2926    [One, Story, 1946, and, Newer, All, Styles]
2927                                 [Split, Foyer]
2928    [One, Story, 1946, and, Newer, All, Styles]
2929                 [Two, Story, 1946, and, Newer]
Name: MSSubClass, Length: 2930, dtype: object

In [ ]:
X_4 = pd.DataFrame()

X_4["MSClass"] = df["MSSubClass"].str.split('_', n = 1, expand = True)[0]

# Check your answer
q_4.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [ ]:
# Lines below will give you a hint or solution code
#q_4.hint()
#q_4.solution()

# 5) Use a Grouped Transform

The value of a home often depends on how it compares to typical homes in its neighborhood. Create a feature `MedNhbdArea` that describes the *median* of `GrLivArea` grouped on `Neighborhood`.
- neighborhood còn dịch là địa điểm của căn nha

In [ ]:
df.groupby(['Neighborhood'])['GrLivArea'].transform('median')

0       1200.0
1       1200.0
2       1200.0
3       1200.0
4       1560.0
         ...  
2925    1282.0
2926    1282.0
2927    1282.0
2928    1282.0
2929    1282.0
Name: GrLivArea, Length: 2930, dtype: float64

In [ ]:
X_5 = pd.DataFrame()

# YOUR CODE HERE
X_5["MedNhbdArea"] = df.groupby(['Neighborhood'])['GrLivArea'].transform('median')

# Check your answer
q_5.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [ ]:
# Lines below will give you a hint or solution code
#q_5.hint()
#q_5.solution()

Now you've made your first new feature set! If you like, you can run the cell below to score the model with all of your new features added:

In [ ]:
X_new = X.join([X_1, X_2, X_3, X_4, X_5])
score_dataset(X_new, y)

0.13954039591355258

In [ ]:
X_new

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,Spaciousness,TotalOutsideSF,Bldg_Duplex,Bldg_OneFam,Bldg_Twnhs,Bldg_TwnhsE,Bldg_TwoFmCon,PorchTypes,MSClass,MedNhbdArea
0,0,0,141.0,31770.0,0,0,0,0,0,0,...,236.571429,272.0,0.0,1656.0,0.0,0.0,0.0,2,0,1200.0
1,0,1,80.0,11622.0,0,0,1,0,0,1,...,179.200000,260.0,0.0,896.0,0.0,0.0,0.0,2,0,1200.0
2,0,0,81.0,14267.0,0,0,0,0,0,0,...,221.500000,429.0,0.0,1329.0,0.0,0.0,0.0,2,0,1200.0
3,0,0,93.0,11160.0,0,0,1,0,0,0,...,263.750000,0.0,0.0,2110.0,0.0,0.0,0.0,0,0,1200.0
4,1,0,74.0,13830.0,0,0,0,0,0,1,...,271.500000,246.0,0.0,1629.0,0.0,0.0,0.0,2,1,1560.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2925,6,0,37.0,7937.0,0,0,0,0,0,2,...,167.166667,120.0,0.0,1003.0,0.0,0.0,0.0,1,2,1282.0
2926,0,0,0.0,8885.0,0,0,0,3,0,1,...,180.400000,164.0,0.0,902.0,0.0,0.0,0.0,1,0,1282.0
2927,4,0,62.0,10441.0,0,0,1,0,0,1,...,161.666667,112.0,0.0,970.0,0.0,0.0,0.0,2,2,1282.0
2928,0,0,77.0,10010.0,0,0,1,0,0,1,...,231.500000,278.0,0.0,1389.0,0.0,0.0,0.0,2,0,1282.0


# Keep Going #

[**Untangle spatial relationships**](https://www.kaggle.com/ryanholbrook/clustering-with-k-means) by adding cluster labels to your dataset.

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/feature-engineering/discussion) to chat with other learners.*